# Crawling Data

In [ ]:
!pip3 install PyGithub requests

     |████████████████████████████████| 291 kB 33.4 MB/s 
     |████████████████████████████████| 961 kB 39.2 MB/s 


In [ ]:
from github import Github

# authenticate to github
g = Github()

repo = g.search_repositories("angular/angular.js")[0]

print("Full name:", repo.full_name)
print("Description:", repo.description)
print("Date created:", repo.created_at)
print("Date of last push:", repo.pushed_at)
print("Home Page:", repo.homepage)
print("Language:", repo.language)
print("Number of forks:", repo.forks)
print("Number of stars:", repo.stargazers_count)
print("-"*50)
# repository content (files & directories)
print("Contents:")
for content in repo.get_contents(""):
    print(content)

Full name: angular/angular.js
Description: AngularJS - HTML enhanced for web apps!
Date created: 2010-01-06 00:34:37
Date of last push: 2021-12-09 23:20:01
Home Page: https://angularjs.org
Language: JavaScript
Number of forks: 28409
Number of stars: 59577
--------------------------------------------------
Contents:
ContentFile(path=".circleci")
ContentFile(path=".editorconfig")
ContentFile(path=".eslintignore")
ContentFile(path=".eslintrc-base.json")
ContentFile(path=".eslintrc-browser.json")
ContentFile(path=".eslintrc-node.json")
ContentFile(path=".eslintrc-todo.json")
ContentFile(path=".eslintrc.json")
ContentFile(path=".gitattributes")
ContentFile(path=".github")
ContentFile(path=".gitignore")
ContentFile(path=".mailmap")
ContentFile(path=".nvmrc")
ContentFile(path="CHANGELOG.md")
ContentFile(path="CODE_OF_CONDUCT.md")
ContentFile(path="CONTRIBUTING.md")
ContentFile(path="DEVELOPERS.md")
ContentFile(path="Gruntfile.js")
ContentFile(path="LICENSE")
ContentFile(path="README.closure.m

In [ ]:
import json

pulls = repo.get_pulls(state='closed', sort='created', direction='asc')

with open('output.json', 'a') as outfile:
  for pr in pulls:
    outfile.write(str(pr))

TypeError: ignored

In [ ]:
usernames_involved = set()

pr = pulls[100]

for review in pr.get_reviews():
    usernames_involved.add(review.user.username)

users_requested, teams_requested = pr.get_review_requests()

for user in users_requested:
    usernames_involved.add(user.username)

for team in teams_requested:
    for user in team.get_members():
        usernames_involved.add(user.username)

print(usernames_involved)

set()


# Implementation

In [4]:
import os
import numpy as np
import shutil
import json
import pandas as pd
import math
from datetime import *
from dateutil import parser
import itertools
from collections import defaultdict
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load and Clean data

In [5]:
# Load data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-07-18 15:43:34', 'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612', 'ownerId': '1', 'reviewersId': ['2'], 'filePaths': ['modules/ssh/files/sshd_config', 'manifests/server.pp', 'modules/ssh/manifests/init.pp']}
{'time': '2011-07-18 16:56:22', 'changeId': 'Iea32ad3aedbee996db53655842061d14c9e3d876', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp', 'modules/jenkins_slave/files/known_hosts']}
{'time': '2011-07-18 17:14:05', 'changeId': 'Ib6af08a6af71a6e5853d958a12f63c381eafa039', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 14:37:53', 'changeId': 'Ide7975c522b6b31a23af67b135135c07d75d9bde', 'ownerId': '2', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/files/pubring.gpg', 'modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 15:12:47', 'changeId': 'I28119ac1735127812866a4b0519368a829e2e666', 'ownerId': '2', 'r

In [6]:
# convert times to datetime format
for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612',
 'filePaths': ['modules/ssh/files/sshd_config',
  'manifests/server.pp',
  'modules/ssh/manifests/init.pp'],
 'ownerId': '1',
 'reviewersId': ['2'],
 'time': datetime.datetime(2011, 7, 18, 15, 43, 34)}

In [7]:
len(data)

6545

In [8]:
# Save only PRs with 1 reviwer (to make the results similar to the paper)
final_data = []
for pr in data:
  if len(pr['reviewersId']) == 1:
    final_data.append(pr)

len(final_data)

3786

## Approach

In [11]:
def get_all_reviewers():
  # create a set of all reviewers
  all_reviewers = set()   
  for pr in final_data:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  return all_reviewers

# all_reviewers = list(get_all_reviewers())

In [9]:
all_reviewers = ['917', '642', '261', '494', '1549', '543', '97', '233', '995', '1179', '1205', '927', '1689', '335', '67', '693', '2170', '1228', '1192', '706', '34', '1155', '838', '1247', '8', '1816', '2230', '845', '2472', '612', '2458', '518', '191', '4', '5', '6', '1216', '512', '385', '679', '7', '1009', '1699', '970', '688', '24', '236', '490', '357', '100', '964', '1', '209', '1030', '2284', '161', '941', '452', '112', '321', '1456', '360', '616', '1132', '330', '979', '2325', '475', '447', '866', '2', '308', '704', '2035', '70', '2243']

print("All Reviewers in project:", all_reviewers)
print("Number of Reviewers in whole project:", len(all_reviewers))

All Reviewers in project: ['917', '642', '261', '494', '1549', '543', '97', '233', '995', '1179', '1205', '927', '1689', '335', '67', '693', '2170', '1228', '1192', '706', '34', '1155', '838', '1247', '8', '1816', '2230', '845', '2472', '612', '2458', '518', '191', '4', '5', '6', '1216', '512', '385', '679', '7', '1009', '1699', '970', '688', '24', '236', '490', '357', '100', '964', '1', '209', '1030', '2284', '161', '941', '452', '112', '321', '1456', '360', '616', '1132', '330', '979', '2325', '475', '447', '866', '2', '308', '704', '2035', '70', '2243']
Number of Reviewers in whole project: 76


In [10]:
# problem hyper-parameters
lambda_val = 7  # can be changed
gama = 0.7

### Time-decay Relationship

In [11]:
def find_PRlist(pull_reqs, reviewerId, cur_time):

  # lambda days before the time of our test PR
  lambda_ago = cur_time - timedelta(days=lambda_val)

  # list of PRs in "pull_reqs" that are between time of lambda days ago up to the time of test PR
  PS = [pr for pr in pull_reqs if ((lambda_ago < pr['time'] < cur_time) and (reviewerId in pr['reviewersId']))]
  # print("size of final list:", len(PS))

  # PS will be the list of PRs which submitted by the owner of test PR and reviewed by any reviewer in lambda days ago
  return PS 

In [12]:
def cal_decayRelation(pull_reqs, cur_time):
  
  return sum([(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

  ######### Alternative #########
  # decayRelationship = 0.0
  # for pr in pull_reqs:
  #   diff_days = (cur_time-pr['time']).days
  #   # print(diff_days)
  #   if diff_days != 0:
  #     decayRelationship += 1/diff_days
  # return decayRelationship

### Time-decay File Similarity

In [13]:
def pathDistance(f1, f2):
  return len(os.path.commonprefix([f1, f2]).split("/")[0:-1]) / max(len(f1.split("/")), len(f2.split("/")))

In [14]:
def prDistance(pr1, pr2):
  
  return sum([pathDistance(f1, f2) for f1 in pr1['filePaths'] for f2 in pr2['filePaths']]) / (len(pr1['filePaths'])*len(pr2['filePaths']))

  ######### Alternative #########
  # path_dist_sum = 0
  # for f1 in pr1['filePaths']:
  #   for f2 in pr2['filePaths']:
  #    # print(pathDistance(f1, f2))
  #     path_dist_sum += pathDistance(f1, f2)
  # return path_dist_sum/(len(pr1['filePaths'])*len(pr2['filePaths']))

In [15]:
def cal_decayFileSimilarity(pull_reqs, cur_time, test_pr):
  
  return sum([prDistance(test_pr, pr)*(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

  ######### Alternative #########
  # decayFileSimilarity = 0.0
  # for pr in pull_reqs:
  #   diff_days = (cur_time-pr['time']).days
  #   # print(diff_days)
  #   if diff_days != 0:
  #     decayFileSimilarity += (prDistance(test_pr, pr) * (1/diff_days))
  # return decayFileSimilarity

### Feature Extraction
Construct a Pull Request Vector for each PR

In [20]:
# For all PRs we should extract PRvector and save this and reviewer ID into a dataframe for future use

vectors = []
labels = []

for t in range(0, len(final_data)):

  prev_pulls = final_data[0:t]
  cur_pull = final_data[t]

  # list of PRs that submitted by the owner of this PR in trainset
  OS = [pr for pr in prev_pulls if (pr['ownerId'] == cur_pull['ownerId'])]

  cur_time = cur_pull['time']

  # Calculating time-decaying Relationship
  PRvector = [cal_decayRelation(find_PRlist(OS, rev, cur_time), cur_time) for rev in all_reviewers]

  # Calculating time-decaying File Similarity
  PRvector.extend([cal_decayFileSimilarity(find_PRlist(prev_pulls, rev, cur_time), cur_time, cur_pull) for rev in all_reviewers])

  if t % 200 == 0:
    print("PR Vector #", t, ": ", PRvector)

  # Save to dataframe
  vectors.append(PRvector)
  labels.append(cur_pull['reviewersId'][0])


print("PR vector size:", len(all_reviewers), "is equal to: 2 *", len(all_reviewers), "=", 2*len(all_reviewers))
print(len(vectors), len(labels))

PR Vector # 0 :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
PR Vector # 200 :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0833333333333333, 2.083333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.37021909827760885, 1.6275031889411005, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
# Create a dataframe of vectors and labels(reviewerIDs)
df = pd.DataFrame()
df['PRvector'] = vectors
df['reviewerID'] = labels
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [23]:
# Save dataframe as csv file
df.to_csv('dataframe.csv')

In [19]:
# Save dataframe into Drive
src = "./dataframe.csv"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/FeatureVectors.csv"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/FeatureVectors.csv'

### Main

In [37]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project/FeatureVectors.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [149]:
# PRs created in the same month should be in the same group
groups = []

start_year = final_data[0]['time'].year
start_month = final_data[0]['time'].month
group = []

for pr in final_data:
  if (pr['time'].year == start_year) and (pr['time'].month == start_month):
    group.append(pr)
    if(pr == final_data[-1]):
      groups.append(group)
  else:
    groups.append(group)
    group = []
    start_year = pr['time'].year
    start_month = pr['time'].month
    group.append(pr)

len(groups)

11

In [ ]:
rounds_num = len(groups)-1

for i in range(0, rounds_num):

  # split train/test sets
  train = groups[0]
  for j in range(1, i+1):
    train.extend(groups[j])
  
  test = groups[i+1]



### Random Forest Classifier

In [106]:
clf = RandomForestClassifier(n_estimators=12, max_depth=None, random_state=0)
# scores = cross_val_score(clf, X, y)
# print(scores.mean())

classifier = clf.fit(X, y)
predictions = classifier.predict_proba(np.reshape(test_PRvector, (1, -1)))
print(predictions)

[[0.58333333 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.16666667 0.         0.         0.
  0.         0.         0.         0.25       0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [91]:
[p for p in range(0, len(predictions[0])) if (predictions[0][p] != 0)]

[0, 20, 27]

In [112]:
[p for p in range(0, len(test_PRvector)) if (test_PRvector[p] != 0)]

[0, 15, 33, 91, 109, 126]

In [113]:
all_reviewers.index('1')

33

### Naive Bayes Classifier

In [111]:
model = GaussianNB()
classifier = model.fit(X, y)
yprob = classifier.predict_proba(np.reshape(test_PRvector, (1, -1)))
print(yprob)

[[5.07134318e-187 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  3.77973295e-189 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 1.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.000000

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [108]:
[p for p in range(0, len(yprob[0])) if (yprob[0][p] != 0)]

[0, 20, 27]

### TRFPre Composer